In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("../RD-systems-and-test-benches/utils")
import model_fsolve as modf
import model_fsolve_v2 as modf2²

import scipy.optimize as sco
import scipy.interpolate as sci

import hx_hydraulic as hxhy
import from_excel as fe
import data_plots as dplt

import fluids as fds
from CoolProp.CoolProp import PropsSI

import openpyxl as opxl
from sklearn.ensemble import RandomForestRegressor

import seaborn as sns
import os
import plotly.graph_objects as go


In [ ]:
path = r"G:\Drive partagés\Cercle Hard\Production\R&D - Modèles numériques\PVT-PL-model\Inputs"
file_name = "\V4.5_1MPE.xlsx"
hw, par, cond = fe.initialize(path, file_name)

In [ ]:
Vdot = 200
Qmax = Vdot/3600000
list_alpha = np.array([0.05, 0.25, 0.5, 0.75, 0.95])
list_Qin = np.array([0.1,0.3,0.5,0.7,0.9])*Qmax
df_testings = fe.testing_series_Qmax(Qmax, list_Qin, list_alpha, par, cond)
df_testings['beta']= round(df_testings['Qin_d']/Qmax, 2)
df_testings['DPd']=df_testings['Pin_d']-df_testings['Pout_d']
df_testings['DPc']=df_testings['Pin_c']-df_testings['Pout_c']
df_testings['DPdc']=df_testings['Pout_d']-df_testings['Pout_c']
df_testings.to_excel(f"G:\\Drive partagés\\Cercle Hard\\Notion\\PRJ-1088 - CFD pour la caractérisation hydraulique d’un échangeur à mini-canaux\\1D_V4.5_MPE_sans calibrage\\Abaque_1MPE_{Vdot}Lh.xlsx")

In [ ]:
import webbrowser
import plotly.graph_objects as go

traces = []
for b in df_testings['beta'].unique():
    df_b = df_testings[df_testings['beta'] == b]
    trace = go.Scatter(x=df_b['alpha'], y=df_b['DPd'], name=f'beta={b}')
    traces.append(trace)

layout = go.Layout(title='DPd vs alpha', xaxis=dict(title='alpha'), yaxis=dict(title='DPd'))
fig1 = go.Figure(data=traces, layout=layout)

traces = []
for b in df_testings['beta'].unique():
    df_b = df_testings[df_testings['beta'] == b]
    trace = go.Scatter(x=df_b['alpha'], y=df_b['DPc'], name=f'beta={b}')
    traces.append(trace)

layout = go.Layout(title='DPc vs alpha', xaxis=dict(title='alpha'), yaxis=dict(title='DPc'))
fig2 = go.Figure(data=traces, layout=layout)

traces = []
for b in df_testings['beta'].unique():
    df_b = df_testings[df_testings['beta'] == b]
    trace = go.Scatter(x=df_b['alpha'], y=df_b['DPdc'], name=f'beta={b}')
    traces.append(trace)

layout = go.Layout(title='DPdc vs alpha', xaxis=dict(title='alpha'), yaxis=dict(title='DPdc'))
fig3 = go.Figure(data=traces, layout=layout)

traces = []
for b in df_testings['alpha'].unique():
    df_b = df_testings[df_testings['alpha'] == b]
    trace = go.Scatter(x=df_b['beta'], y=df_b['DPd'], name=f'alpha={b}')
    traces.append(trace)

layout = go.Layout(title='DPd vs beta', xaxis=dict(title='beta'), yaxis=dict(title='DPd'))
fig4 = go.Figure(data=traces, layout=layout)

traces = []
for b in df_testings['alpha'].unique():
    df_b = df_testings[df_testings['alpha'] == b]
    trace = go.Scatter(x=df_b['beta'], y=df_b['DPc'], name=f'alpha={b}')
    traces.append(trace)

layout = go.Layout(title='DPc vs beta', xaxis=dict(title='beta'), yaxis=dict(title='DPc'))
fig5 = go.Figure(data=traces, layout=layout)

traces = []
for b in df_testings['alpha'].unique():
    df_b = df_testings[df_testings['alpha'] == b]
    trace = go.Scatter(x=df_b['beta'], y=df_b['DPdc'], name=f'alpha={b}')
    traces.append(trace)

layout = go.Layout(title='DPdc vs beta', xaxis=dict(title='beta'), yaxis=dict(title='DPdc'))
fig6 = go.Figure(data=traces, layout=layout)


with open(f'abaque1D_1riser_{Vdot}Lh.html', 'w') as f:
    f.write('')
with open(f'abaque1D_1riser_{Vdot}Lh.html', 'a') as f:
    f.write(fig1.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(fig2.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(fig3.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(fig4.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(fig5.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(fig6.to_html(full_html=False, include_plotlyjs='cdn'))

    file_path = f'abaque1D_1riser_{Vdot}Lh.html'
    webbrowser.open(file_path)


In [ ]:
import time

def group_by(tabl, N_groups):
    tabl_group = pd.DataFrame({'Pin':[], 'Pout':[],'Q':[]})
    N_risers = len(tabl)//N_groups
    for i in range(N_groups):
        tabl_group.loc[len(tabl_group)] = [tabl.loc[i*N_risers]['Pin'], tabl.loc[(i+1)*N_risers - 1]['Pout'], np.sum(tabl.loc[i*N_risers:(i+1)*N_risers-1,'qx'])]
    return tabl_group

def v1_generator(Vdot_list, N_list, name):
    for N in N_list:
        path = r"G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Inputs"
        file_name = f"\{name}_x{N}.xlsx"
        hw, par, cond = fe.initialize(path, file_name)
        for Vdot in Vdot_list:
            Vdot = int(Vdot)
            cond['Dv'] = Vdot/3600000
            t1=time.time()
            tabl, res, PL, residuals = modf.PL_fsolve(par, cond)
            tabl.to_excel(f'G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Outputs\{name}_x{N}\{name}_x{N}_{Vdot}Lh.xlsx')
            tabl_group = group_by(tabl, N)
            tabl_group.to_excel(f'G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Outputs\{name}_x{N}\{name}_x{N}_{Vdot}Lh_group.xlsx')
            t2 = time.time()
            print(Vdot, N, f'{t2-t1 :2f} secondes')

def v2_generator(Vdot_list, N_list, name, path):
    for N in N_list:
        for Vdot in Vdot_list:
            Vdot = int(Vdot)
            df_testings = pd.read_excel(f'G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Outputs\Tests-model-simplification\{name}_test_{Vdot}Lh.xlsx')
            DPin, DPout, DPx = modf2.transfer_func(df_testings)
            tabl_panel, f = modf2.PL_fsolve_MPE(N, Vdot/3600000, DPin, DPout, DPx)
            tabl_panel.to_excel(f'G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Outputs\{name}_x{N}_simplified\{name}_x{N}_{Vdot}Lh.xlsx')
            df_residuals = pd.DataFrame({'residuals':f})
            df_residuals.to_excel(f'G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Outputs\{name}_x{N}_simplified\{name}_x{N}_{Vdot}Lh_residuals.xlsx')



In [ ]:
from scipy import interpolate
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go

df_abaque = pd.read_excel(r"G:\Drive partagés\Cercle Hard\Notion\PRJ-1088 - CFD pour la caractérisation hydraulique d’un échangeur à mini-canaux\CFD_V4.5_1MPE_woch\AbaqueCFD_1MPE_woch.xlsx")

traces_PL = []
all_residuals = []
all_tabl = []

for N in range(1,4):
    list_tabl = []
    list_residuals = []
    list_PL = []
    for Vdot in df_abaque['Vdot_max'].unique():
        df_fil = df_abaque[df_abaque['Vdot_max'] == Vdot]
        X1_name = 'alpha'
        X2_name = 'beta'
        X_columns = df_fil[[X1_name, X2_name]]
        poly = PolynomialFeatures(degree=3, include_bias=False)
        X_poly = poly.fit_transform(X_columns)

        model_d = LinearRegression()
        model_d.fit(X_poly, df_fil['DPd'])
        model_c = LinearRegression()
        model_c.fit(X_poly, df_fil['DPc'])
        model_dc = LinearRegression()
        model_dc.fit(X_poly, df_fil['DPdc'])

        # def DPd(Qin, alpha):
        #     point = np.array([[Qin*3600000/Vdot, alpha]])
        #     X_poly = poly.fit_transform(point)
        #     return model_d.predict(X_poly)[0]
        # def DPc(Qin, alpha):
        #     point = np.array([[Qin*3600000/Vdot, alpha]])
        #     X_poly = poly.fit_transform(point)
        #     return model_c.predict(X_poly)[0]
        # def DPdc(Qin, alpha):
        #     point = np.array([[Qin*3600000/Vdot, alpha]])
        #     X_poly = poly.fit_transform(point)
        #     return - model_dc.predict(X_poly)[0]
        DPd, DPc, DPdc = modf2.transfer_func(df_fil, deg_a = 2, deg_q = 2)
        tabl, residuals = modf2.PL_fsolve_MPE(N*19, Vdot/3600000, DPd, DPc, DPdc)
        list_tabl.append(tabl)
        list_PL.append(tabl['Pin'][0])
        list_residuals.append(residuals)

    all_residuals.append(list_residuals)
    all_tabl.append(list_tabl)
    list_Vdot = df_abaque['Vdot_max'].unique()
    traces = []
    for i, tabl in enumerate(list_tabl):
        trace = go.Scatter(x=tabl.index, y=tabl['qx'], name=f'Vdot = {list_Vdot[i]}')
        traces.append(trace)

    layout = go.Layout(title=f'{N} panneau(x)', xaxis=dict(title='MPE'), yaxis=dict(title='qx'))
    fig = go.Figure(data=traces, layout=layout)
    fig.show()

    trace_PL = go.Scatter(x=list_Vdot, y=list_PL, name=f'N = {N} panneaux')
    traces_PL.append(trace_PL)

layout = go.Layout(title='PL vs Vdot', xaxis=dict(title='Vdot'), yaxis=dict(title='PL'))
fig = go.Figure(data=traces_PL, layout=layout)
fig.show()

In [ ]:
for N, list_tabl in enumerate(all_tabl):
    traces = []
    for i, tabl in enumerate(list_tabl):
        trace = go.Scatter(x=tabl.index, y=tabl['qx'], name=f'Vdot = {list_Vdot[i]}L/h')
        traces.append(trace)
    layout = go.Layout(title=f'{N+1} panneau(x)', xaxis=dict(title='MPE'), yaxis=dict(title='qx [L/h]'))   
    fig = go.Figure(data=traces, layout=layout)
    fig.show()


In [ ]:
list_rmse = []
for Vdot in [100,200,400,600]:
    df_fil = df_abaque[df_abaque['Vdot_max'] == Vdot]
    DPd, DPc, DPdc = modf2.transfer_func(df_fil, deg_a = 2, deg_q = 2)
    alphas = np.array(df_fil['alpha'])
    betas = np.array(df_fil['beta'])
    DPds = np.array([DPd(betas[i]*Vdot/3600000, alphas[i]) for i in range(len(alphas))])
    actual_DPd = np.array(df_fil['DPd'])
    DPcs = np.array([DPc(betas[i]*Vdot/3600000, alphas[i]) for i in range(len(alphas))])
    actual_DPc = np.array(df_fil['DPc'])
    DPdcs = np.array([DPdc(betas[i]*Vdot/3600000, alphas[i]) for i in range(len(alphas))])
    actual_DPdc = -np.array(df_fil['DPdc'])
    list_rmse.append([np.sqrt(mean_squared_error(actual_DPd, DPds)), np.sqrt(mean_squared_error(actual_DPc, DPcs)), np.sqrt(mean_squared_error(actual_DPdc, DPdcs))])

list_rmse

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import plotly.graph_objects as go

df_fil = df_abaque[df_abaque['Vdot_max'] == 600]
Y_name = 'DPc'
X1_name = 'alpha'
X2_name = 'beta'
X_columns = df_fil[[X1_name, X2_name]]
# Generate polynomial features up to the third degree
poly = PolynomialFeatures(degree=3, include_bias=False)
X_poly = poly.fit_transform(X_columns)
# Fit the model
model = LinearRegression()
model.fit(X_poly, df_fil[Y_name])
# Coefficients
coefficients = model.coef_
intercept = model.intercept_
# Generate a grid for plotting
x1_range = np.linspace(df_fil[X1_name].min(), df_fil[X1_name].max(), 50)
x2_range = np.linspace(df_fil[X2_name].min(), df_fil[X2_name].max(), 50)
x1_grid, x2_grid = np.meshgrid(x1_range, x2_range)
# Flatten the grid to apply transformations
X_grid = np.vstack([x1_grid.ravel(), x2_grid.ravel()]).T
# Generate polynomial features for the grid
X_grid_poly = poly.transform(X_grid)
# Predict Y values over the grid
y_pred_grid = model.predict(X_grid_poly).reshape(x1_grid.shape)
# Step 1: Generate a grid
x1_range = np.linspace(df_fil[X1_name].min(), df_fil[X1_name].max(), 100)
x2_range = np.linspace(df_fil[X2_name].min(), df_fil[X2_name].max(), 100)
x1_grid, x2_grid = np.meshgrid(x1_range, x2_range)
# Flatten the grid to apply transformations
X_grid = np.vstack([x1_grid.ravel(), x2_grid.ravel()]).T
# Generate polynomial features for the grid
X_grid_poly = poly.transform(X_grid)
# Predict Y values over the grid
y_pred_grid = model.predict(X_grid_poly).reshape(x1_grid.shape)
# Step 3: Plot the surface
fig = go.Figure()
fig.add_trace(go.Surface(x=x1_grid, y=x2_grid, z=y_pred_grid, name='Interpolated Surface', colorscale='Viridis'))
# Step 4: Overlay the original data points
fig.add_trace(go.Scatter3d(x=df_fil[X1_name], y=df_fil[X2_name], z=df_fil[Y_name], mode='markers',
                           marker=dict(size=5, color='red'), name='Original Data'))
# Update plot layout
fig.update_layout(title='3D Surface and Points Plot', autosize=False,
                  width=700, height=700,
                  margin=dict(l=65, r=50, b=65, t=90),
                  scene=dict(
                      xaxis_title=X1_name,
                      yaxis_title=X2_name,
                      zaxis_title=Y_name))
# Show the plot
fig.show()
# Step 1: Use the model to predict Y values for the original data points
Y_pred = model.predict(X_poly)  # X_poly contains the polynomial features of the original data
# Step 2: Calculate the differences (errors) between predicted and actual Y values
Y_actual = df_fil[Y_name].values  # Actual Y values from the DataFrame
# Step 3: Compute the RMSE
rmse = np.sqrt(mean_squared_error(Y_actual, Y_pred))
print(f"RMSE: {rmse}")

In [ ]:
DPdc(0.7*Vdot/3600000, 0.5)

In [ ]:
import plotly.graph_objects as go
import numpy as np

x1 = np.arange(0, 1, 0.01) * 600 / 3600000
x2 = np.arange(0, 1, 0.01)

# Create a meshgrid from x1 and x2
X1, X2 = np.meshgrid(x1, x2)

# Calculate the values of DPd for each combination of x1 and x2
Z = np.zeros_like(X1)
for i in range(len(x1)):
    for j in range(len(x2)):
        Z[j, i] = -DPdc(X1[j, i], X2[j, i])

# Create a 3D surface plot
fig = go.Figure(data=[go.Surface(x=X1, y=X2, z=Z)])

# Set labels and title
fig.update_layout(scene=dict(
    xaxis_title='x1',
    yaxis_title='x2',
    zaxis_title='DPdc'),
    title='3D Surface Plot')


fig = go.Figure(data=[go.Surface(x=X1, y=X2, z=Z)])

# Add the scatter plot of DPd vs alpha and beta
fig.add_trace(go.Scatter3d(x=df_testings['beta']*600/3600000, y=df_testings['alpha'], z=df_testings['DPdc'],
                           mode='markers', marker=dict(size=5), name='DPdc'))

# Set labels and title
fig.update_layout(scene=dict(
    xaxis_title='Qin_d',
    yaxis_title='alpha',
    zaxis_title='DPdc'),
    title='3D Surface and Scatter Plot')

fig.update_layout(title='3D Surface and Points Plot', autosize=False,
                width=700, height=700,
                margin=dict(l=65, r=50, b=65, t=90))

# Show the plot
fig.show()



In [ ]:
import seaborn as sns

N_list = range(2, 6)
name = 'V4.6'
Vdot_list = np.arange(50, 1050, 50)

def heatmap_error(N_list, Vdot_list):
    df_N = pd.DataFrame({'N':N_list})
    df_Vdot = pd.DataFrame({'Vdot':Vdot_list})
    df_error =df_N.merge(df_Vdot, how='cross')
    df_error['flow rate error'] = np.nan
    df_error['pressure error'] = np.nan
    df_error['residuals v2'] = np.nan
    for n in N_list:
        for Vdot in Vdot_list:
            tabl1 = pd.read_excel(f'G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Outputs\{name}_x{n}\{name}_x{n}_{Vdot}Lh_group.xlsx')
            tabl2 = pd.read_excel(f'G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Outputs\{name}_x{n}_simplified\{name}_x{n}_{Vdot}Lh.xlsx')
            q = np.mean(np.abs((tabl1['Q'] - tabl2['qx'])*100/tabl1['Q']))
            p = np.abs((tabl1['Pin'][0] - tabl2['Pin'][0])*100/tabl1['Pin'][0])
            df_error.loc[(df_error['N'] == n) & (df_error['Vdot'] == Vdot), 'flow rate error'] = q
            df_error.loc[(df_error['N'] == n) & (df_error['Vdot'] == Vdot), 'pressure error'] = p
            df_residuals = pd.read_excel(f'G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Outputs\{name}_x{n}_simplified\{name}_x{n}_{Vdot}Lh_residuals.xlsx')
            df_error.loc[(df_error['N'] == n) & (df_error['Vdot'] == Vdot), 'residuals v2'] = np.max(np.abs(df_residuals['residuals']))
    cmap = sns.diverging_palette(120, 10, as_cmap=True)
    sns.heatmap(df_error.pivot(index='N', columns='Vdot', values='flow rate error'), annot=True, fmt=".0f", cmap=cmap, vmin=0, vmax=20)
    plt.title(f'{name} flow rate error [%]')
    plt.show()

    sns.heatmap(df_error.pivot(index='N', columns='Vdot', values='pressure error'), annot=True, fmt=".0f", cmap=cmap, vmin=0, vmax=20)
    plt.title(f'{name} pressure loss error [%]')
    plt.show()

    sns.heatmap(df_error.pivot(index='N', columns='Vdot', values='residuals v2'), annot=True, fmt=".0f", cmap=cmap, vmin=0, vmax=20)
    plt.title(f'{name} residuals v2')
    plt.show()


heatmap_error(N_list, Vdot_list)

In [ ]:
list_v_n = []
list_residuals = []
for n in N_list:
    for Vdot in Vdot_list:
        df_residuals = pd.read_excel(f'G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Outputs\{name}_x{n}_simplified\{name}_x{n}_{Vdot}Lh_residuals.xlsx')
        list_v_n.append(Vdot/n)
        list_residuals.append(np.max(np.abs(df_residuals['residuals'])))

plt.plot(list_v_n, list_residuals, 'o')
plt.xlabel('Vdot/N')
plt.ylabel('max residuals')
plt.show()

In [ ]:
import os

list_D_man = np.arange(10, 101, 15)*10**-3
list_D_riser = np.array([1, 5, 10, 25, 50])*10**-3
list_N = np.array([2, 10, 50, 100, 300])
L_inter_riser = np.array([0.1, 0.5, 2, 10, 30])*10**-3
list_L_riser = np.array([0.2, 0.5, 1, 2])
N_panels_list = np.array([2, 3, 4])
Vdot_list = np.array([200, 400, 600])

def PROOF_OF_CONCEPT(list_D_man, list_D_riser, list_N, L_inter_riser, list_L_riser):
    hx, par, cond = fe.initialize('G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Inputs', "\MICOE.xlsx")
    par['N_EP'] = 1
    par['shape_man'] = 'tubular'
    par['shape_riser'] = 'tubular'
    for D_man in list_D_man:
        for D_riser in list_D_riser:
            for N in list_N:
                for L_inter in L_inter_riser:
                    for L_riser in list_L_riser:
                        if D_man > D_riser:
                            par["N"] = N
                            par["ref"] = N-1
                            par['D_man'] = D_man
                            par['A_man'] = np.pi*(D_man/2)**2
                            par['D_riser'] = D_riser
                            par['A_riser'] = np.pi*(D_riser/2)**2
                            par['N_riser_per_panel'] = N
                            par['N_per_EP'] = N
                            par['inter_riser'] = L_inter
                            par['Ly']= [L_inter]*(N-1)
                            par['L_man'] = L_inter*N
                            par['L_riser'] = L_riser
                            path = f"C:\\Users\\DualSun\\Desktop\\POC\\Man_{D_man}_Riser_{D_riser}_N_{N}_L_inter_{L_inter}_L_riser_{L_riser}"
                            os.mkdir(path)
                            for Vdot in Vdot_list:
                                Qmax = Vdot/3600000
                                df_testings = fe.testing_series_Qmax(Qmax, np.array([0.1, 0.3, 0.5, 0.8, 1.])*Qmax, np.array([0.1, 0.3, 0.5, 0.8, 1.]), par, cond, to_excel=False, file_name=None)
                                df_testings.to_excel(path+f'\Abaque_{Vdot}Lh.xlsx')
                            
                            for N_panel in N_panels_list:
                                for Vdot in Vdot_list:
                                    Vdot = int(Vdot)
                                    df_testings = pd.read_excel(path+f'\Abaque_{Vdot}Lh.xlsx')
                                    DPin, DPout, DPx = modf2.transfer_func(df_testings)
                                    tabl_panel, f = modf2.PL_fsolve_MPE(N, Vdot/3600000, DPin, DPout, DPx)
                                    tabl_panel.to_excel(path+f'\X{N_panel}_{Vdot}Lh_v2.xlsx')
                                    df_residuals = pd.DataFrame({'residuals':f})
                                    df_residuals.to_excel(path+f'\X{N_panel}_{Vdot}Lh_residuals_v2.xlsx')

                                    par['N_panel'] = N_panel
                                    cond['Dv'] = Vdot/3600000
                                    tabl_v1, res, PL, residuals = modf.PL_fsolve(par, cond)
                                    tabl_v1.to_excel(path+f'\X{N_panel}_{Vdot}Lh_v1.xlsx')
                                    tabl_group = group_by(tabl_v1, N)
                                    tabl_group.to_excel(path+f'\X{N_panel}_{Vdot}Lh_group_v1.xlsx')
                                    df_residuals = pd.DataFrame({'residuals':residuals})
                                    df_residuals.to_excel(path+f'\X{N_panel}_{Vdot}Lh_residuals_v1.xlsx')

                    
PROOF_OF_CONCEPT(list_D_man, list_D_riser, list_N, L_inter_riser, list_L_riser)

In [ ]:
path = r"G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Inputs"
file_name = "\V4.5_coeff.xlsx"
hw, par, cond = fe.initialize(path, file_name)
list_Qmax = np.arange(650,1050, 50)/3600000
for Qmax in list_Qmax:
    list_Qin = np.arange(0.1, 1.1, 0.1)*Qmax
    list_alpha = np.arange(0.05, 1.05, 0.05)
    df_testings = fe.testing_series_Qmax(Qmax, list_Qin, list_alpha, par,cond)
    df_testings.to_excel(r"G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Outputs\Tests-model-simplification" + file_name[:-5]+f"_test_{int(Qmax*3600000)}Lh.xlsx", index=False)
    print(Qmax)

In [ ]:
##courbe de pertes de charge

path = r"G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Inputs"
file_name = "\V4.5_1MPE.xlsx"
hw, par, cond = fe.initialize(path, file_name)

list_PL = []
list_PL_2 = []
list_Qmax = np.array([50, 100, 150, 200])/3600000
for Qmax in list_Qmax:
    list_Qin = np.arange(0.1, 1.1, 0.05)*Qmax
    list_alpha = np.arange(0.1, 1.1, 0.05)
    df_testings = fe.testing_series_Qmax(Qmax, list_Qin, list_alpha, par,cond)
    DPin, DPout, DPx = modf.transfer_func(df_testings)
    tabl, f = modf2.PL_fsolve_MPE(19, Qmax, DPin, DPout, DPx)
    list_PL.append(tabl['Pin'][0])

    cond['Dv'] = Qmax
    tabl, res, PL, testings = modf.PL_fsolve(par, cond)
    list_PL_2.append(res)

plt.plot(list_Qmax*3600000, list_PL, label = 'MPE')
plt.plot(list_Qmax*3600000, list_PL_2, label = 'riser')
plt.xlabel('Qmax [L/h]')
plt.ylabel('PL [Pa]')
plt.legend()
plt.show()

In [ ]:
df_abaque = pd.read_excel(r"G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Outputs\Tests-model-simplification\\AbaqueCFD_V4.5_MPE_x1.xlsx")
df_abaque = df_abaque[df_abaque['Q_max'] <= 0.027778]
df_abaque

In [ ]:
# plot le nuage de points df_abaque['DP_dc'] en fonction de alpha, coloré par Qin_d
df_abaque['DPc']=df_abaque['Pin_c']-df_abaque['Pout_c']
df_abaque['DPd']=df_abaque['Pin_d']-df_abaque['Pout_d']
df_abaque['DPdc']=df_abaque['Pout_d']-df_abaque['Pout_c']
sns.relplot(data=df_abaque, x='alpha', y='DPd', hue='Qin_d', aspect=1.61)

# superposer cette courbe avec la courbe théorique DPc(alpha) pour chaque Qin_d
DPd, DPc, DPdc = modf2.transfer_func(df_abaque, type='polynomial', deg_a=3, deg_q=4)

alpha = np.arange(0.05, 1.05, 0.05)
Qin_d = np.arange(0.1, 1., 0.2)*100/3600000
for q in Qin_d:
    DP_list = [DPd(q, a) for a in alpha]
    plt.plot(alpha, DP_list, label=f'Qin_d = {q}')
plt.legend()
plt.show()


In [ ]:
df_testings = pd.read_excel(r"G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Outputs\Tests-model-simplification\V4.5_1MPE_test.xlsx")
# plots df_testings.plot(x='alpha', y = 'Pin collect') colored by Qin
sns.relplot(data=df_testings, x='alpha', y='Pin_c', hue='Qin distrib', aspect=1.61)
# plt.plot(list_alpha, [Pin(2.5e-5, alpha) for alpha in list_alpha])
plt.show()

In [ ]:
file_name = "\V4.5.xlsx"
hw, par, cond = fe.initialize(path, file_name)
tabl, res, PL, testings, residuals = modf.PL_fsolve(par, cond)

In [ ]:
N = 6
name = 'V4.6'
Vdot = 600

tabl_group = pd.read_excel(f"G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Outputs\{name}_x{N}\{name}_x{N}_{Vdot}Lh_group.xlsx", index_col=0)
tabl_2 = pd.read_excel(f"G:\Drive partagés\Cercle Hard\Innovation\PVT-PL-model\Outputs\{name}_x{N}_simplified\{name}_x{N}_{Vdot}Lh.xlsx", index_col=0)
plt.plot(np.arange(0,len(tabl_group)),tabl_group['Q'], label = 'method = riser')
plt.plot(np.arange(0,len(tabl_2)), tabl_2['qx'], label = 'method = panel')
plt.xlabel('panel')
plt.ylabel('flow rate [L/h]')
plt.title(f'{N} panels {name}, {Vdot} L/h')
plt.legend()
plt.show()